In [1]:
import os
from google.colab import drive

# Mount drive and change directory
drive.mount('/content/drive')

path = 'NLP Project'
os.chdir(f'/content/drive/MyDrive/{path}')

Mounted at /content/drive


In [39]:
# Install needed packages
!pip install SpeechRecognition
!pip install pydub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for punctuator: filename=punctuator-0.9.6-py3-none-any.whl size=25134 sha256=49ff2c82c99f6d81cdfd3cbc4d5d7a367996f5d03000ac67312c88f4b627e308
  Stored in directory: /root/.cache/pip/wheels/95/ac/19/4b7f3e91a1d41172b078bbebd32696e732cf3a26b1a93d0f1a
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668109 sha256=bfd3996f509f8b61b3e8e54eb96aece078493401ee80b3ab6446594740e1605d
  Stored in directory: /root/.cache/pip/wheels/d9/e6/7d/2267d21a99e4ab8276f976f293b4ff23f50c9d809f4a216ebb
Successfully built punctuator theano


## Transcribe the audio files

In [ ]:
from pydub import AudioSegment, silence
import speech_recognition as sr
import numpy as np
from itertools import zip_longest

In [62]:
def split_audio_on_silence(audio_path, min_silence_len=2500):
    """Load an audio file and split on silences."""
    audio = AudioSegment.from_wav(audio_path)
    # This may need adjustment based on the specific audio file characteristics
    return silence.split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=audio.dBFS-15, keep_silence=100)

def transcribe_segments(segments):
    """Transcribe a list of AudioSegment objects."""
    recognizer = sr.Recognizer()
    transcriptions = []

    for i, segment in enumerate(segments):
        # Export segment to a temporary WAV file
        segment.export("temp_segment.wav", format="wav")

        # Recognize the audio
        with sr.AudioFile("temp_segment.wav") as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio_data = recognizer.record(source)
            try:
                transcription = recognizer.recognize_google(audio_data, language='en-US', show_all=False)
                transcriptions.append(transcription)
            except sr.UnknownValueError:
                print(f"Segment {i+1}: Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Segment {i+1}: Could not request results; {e}")

    return transcriptions

def main(audio_file):
    segments = split_audio_on_silence(audio_file)
    transcriptions = transcribe_segments(segments)
    return transcriptions

In [63]:
# Not that good, something we have to work on

# Paths to files
agent_file = 'gridspace-stanford-harper-valley/data/audio/agent/0a6f57765f4641a2.wav'
caller_file = 'gridspace-stanford-harper-valley/data/audio/caller/0a6f57765f4641a2.wav'

agent_transcription = main(agent_file)
caller_transcription = main(caller_file)

print('Conversation\n')
for agent, caller in zip_longest(agent_transcription, caller_transcription, fillvalue="???"):
    if agent:
        print(f'Agent: {agent}')
    if caller:
        print(f'Caller: {caller}')

Segment 6: Google Speech Recognition could not understand audio
Segment 5: Google Speech Recognition could not understand audio
Conversation

Agent: hello Google National Bank my name is Mary how can I help you
Caller: hi my name is Patricia Jones I would like to transfer money between my
Agent: what is the telephone
Caller: the amount is $102
Agent: and what is the social
Caller: from my checking account to my savings
Agent: $102 has been transferred from your checking account to your savings account is there anything else I can help you
Caller: thank you know that was all thank you
Agent: okay thank you for calling have a great day
Caller: ???
